# Embeddings Analysis
This notebook contains the code for segmenting, visualizing and analysing the deep features.

In [ ]:
import os
import datetime
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from IPython.display import Markdown, display

In [ ]:
def load_embeddings(subject):
    path = '../embeddings/embeddings_' + subject
    normal_embs = np.load(path + '_normal.npy')
    sleepy_embs = np.load(path + '_sleepy.npy')

    return normal_embs, sleepy_embs

In [ ]:
# Extracts average embedding in a segment from given embeddigs
def embeddings_segment(embeddings, video_len, segment_len):
    # The amount of frames at the end that are not taken into account
    rest = video_len % segment_len
    num_frames = video_len - rest
    avg_embeddings = []
    acum_embeddings = np.zeros(2048)
    # a blink is counted to a segment,when the blink starts in that segment
    for frame in range(num_frames + 1):
        # if frame % 1000 == 0:
        #     print(frame)
        acum_embeddings = acum_embeddings + np.array(embeddings[frame])
        # only happens at the end of a segment
        if frame % segment_len == 0 and frame != 0:
            avg_embeddings.append(acum_embeddings / segment_len)
            acum_embeddings = np.zeros(2048)
            #print('New segment: ', frame)
    normalized_embeddings = preprocessing.normalize(avg_embeddings, norm='l2')
    return normalized_embeddings

In [ ]:
subject = 'sub23'
norm_embs, sleep_embs = load_embeddings(subject)

print(len(norm_embs), len(sleep_embs))

segment_length = int(46 * 60 * 1)
avg_norm_embs = embeddings_segment(norm_embs, len(norm_embs), segment_length)
avg_sleep_embs = embeddings_segment(sleep_embs, len(sleep_embs), segment_length)

In [ ]:
norm_len = len(avg_norm_embs)
sleep_len = len(avg_sleep_embs)
all_embs = list(avg_norm_embs) + list(avg_sleep_embs)

print(norm_len, sleep_len)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pri_comps = pca.fit_transform(all_embs)
var = pca.explained_variance_ratio_
print('Percentage of variance explained by the two components: ', float(sum(var)))
pc_normal = pri_comps[:norm_len]
pc_sleep = pri_comps[norm_len:]

In [ ]:
plt.scatter(pc_normal[:,0], pc_normal[:,1], label = 'Normal')
plt.scatter(pc_sleep[:,0], pc_sleep[:,1], label = 'Sleep Restricted')
#plt.scatter(pri_comps[:,0], pri_comps[:,1])

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('The first two principal components of 1 min segments of subject '+ subject[-2:])
plt.legend()
plt.savefig('../final_figures/pc_plot_subject'+ subject[-1] + '.jpg')
plt.show()

## Check Wilcoxon across all subjects

In [ ]:
from scipy import stats
def wilcoxon_test(normal_vals, sleepy_vals):
    w_score = stats.wilcoxon(normal_vals, sleepy_vals[:len(normal_vals)])
    return w_score[0], w_score[1]


def equally_sized_lists(list1, list2):
    if len(list1) > len(list2):
        list1 = list1[:len(list2)]
    elif len(list1) < len(list2):
        list2 = list2[:len(list1)]
    return list1, list2

def wilcoxon_check(subjects):
    sum_score = 0
    for sub in subjects:
        subject = 'sub' + str(sub)
        norm_embs, sleep_embs = load_embeddings(subject)
        avg_norm_embs = embeddings_segment(norm_embs, len(norm_embs), 46 * 60)
        avg_sleep_embs = embeddings_segment(sleep_embs, len(sleep_embs), 46 * 60)
        norm_len = len(avg_norm_embs)

        all_embs = list(avg_norm_embs) + list(avg_sleep_embs)
        pca = PCA(n_components=2)
        pri_comps = pca.fit_transform(all_embs)

        pc_normal = pri_comps[:norm_len]
        pc_sleepy = pri_comps[norm_len:]

        pc_norm, pc_sleep = equally_sized_lists(np.concatenate((pc_normal[:,0], pc_normal[:,1])), np.concatenate((pc_sleepy[:,0], pc_sleepy[:,1])))

        w_score = wilcoxon_test(pc_norm, pc_sleep)
        sum_score += w_score[1]
        
        print("Done with subject " + str(sub))

    return (sum_score / len(subjects))

subjects = [1,3,4,6,7,9,10,11,12,14,15,16,20,22,23,24,25]
avg_wil = wilcoxon_check(subjects)
print("Average wilcoxon p-value for the first two principal components combined: ", avg_wil)

## Check PCA Var across all subjects

In [ ]:
def pca_var_check(subjects):
    sum_var = 0
    for sub in subjects:
        subject = 'sub' + str(sub)
        norm_embs, sleep_embs = load_embeddings(subject)
        avg_norm_embs = embeddings_segment(norm_embs, len(norm_embs), 46 * 60)
        avg_sleep_embs = embeddings_segment(sleep_embs, len(sleep_embs), 46 * 60)
        all_embs = list(avg_norm_embs) + list(avg_sleep_embs)
        pca = PCA(n_components=9)
        pri_comps = pca.fit_transform(all_embs)
        var = pca.explained_variance_ratio_
        sum_var += sum(var)
    return sum_var / len(subjects)

subjects = [1,3,4,6,7,9,10,11,12,14,15,16,20,22,23,24,25]
avg_var = pca_var_check(subjects)
print("Average percentage of captured variance across all subjects: ", int(avg_var * 100))

## Classification

In [ ]:
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def train_model(X_normal, X_sleepy):
    y_normal = list(np.zeros(len(X_normal)))
    y_sleepy = list(np.ones(len(X_sleepy)))

    X = X_normal + X_sleepy
    y = y_normal + y_sleepy

    scaler = MinMaxScaler()
    X_norm = scaler.fit_transform(X)
    #print("Maximum values :", scaler.data_max_)
    #print("Minimum values :", scaler.data_min_)

    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3,random_state=42)
    
    # train model
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    f1_round =  round(f1, 2) 
    print("F1 score: ", f1_round)


    return model, f1_round, X_train, X_test, y_train, y_test


X_normal = list(zip(pc_normal[:,0], pc_normal[:,1]))
X_sleepy = list(zip(pc_sleep[:,0], pc_sleep[:,1]))

model, f1_round, X_train, X_test, y_train, y_test = train_model(X_normal, X_sleepy)

In [ ]:
def class_subject_seperately(subjects, seg_len_list):
    avg_f1_scores = []
    for seg_len in seg_len_list:
        seg_len = int(46 * 60 * seg_len)
        f1_scores = []
        for sub in subjects:
            subject = 'sub' + str(sub)
            norm_embs, sleep_embs = load_embeddings(subject)
            avg_norm_embs = embeddings_segment(norm_embs, len(norm_embs), seg_len)
            avg_sleep_embs = embeddings_segment(sleep_embs, len(sleep_embs), seg_len)
            norm_len = len(avg_norm_embs)
            sleep_len = len(avg_sleep_embs)
            all_embs = list(avg_norm_embs) + list(avg_sleep_embs)
            pca = PCA(n_components=2)
            pri_comps = pca.fit_transform(all_embs)
            pc_normal = pri_comps[:norm_len]
            pc_sleepy = pri_comps[norm_len:]

            X_normal = list(zip(pc_normal[:,0], pc_normal[:,1]))
            X_sleepy = list(zip(pc_sleepy[:,0], pc_sleepy[:,1]))

            model, f1_round, X_train, X_test, y_train, y_test = train_model(X_normal, X_sleepy)

            f1_scores.append(f1_round)

            print("Subject " + str(sub)+ " is done")
        print("Average F1 score of segment length " + str(seg_len), sum(f1_scores) / len(f1_scores))
        avg_f1_scores.append(sum(f1_scores) / len(f1_scores))
    return avg_f1_scores
# 6,9 16, 20 dont have enough sleepy segments for seg len is 5
subjects = [1,3,4,10,11,12,14,15,22,23,24]

seg_len_list = [0.11, 0.5, 1, 2, 5]
avg_f1_scores = class_subject_seperately(subjects, seg_len_list)
print(avg_f1_scores)

x_values = range(len(seg_len_list))
plt.plot(x_values, avg_f1_scores)
plt.xticks(x_values, seg_len_list)
plt.xlabel('Segment length (in minutes)')
plt.ylabel('F1 score')
plt.title('Deep classification performance for all subjects when increasing segment length')
plt.savefig('../final_figures/deep_classification.jpg')
plt.show()

In [ ]:
import random
from collections import Counter

def obtain_data(subjects, pca):
    seg_len = int(46 * 60 * 1)
    first = True
    for sub in subjects:
        subject = 'sub' + sub
        if first:
            norm_embs, sleep_embs = load_embeddings(subject)
            avg_norm_embs = embeddings_segment(norm_embs, len(norm_embs), seg_len)
            avg_sleep_embs = embeddings_segment(sleep_embs, len(sleep_embs), seg_len)
            if pca:
                norm_len = len(avg_norm_embs)
                sleep_len = len(avg_sleep_embs)
                all_embs = list(avg_norm_embs) + list(avg_sleep_embs)
                pca = PCA(n_components=3)
                pri_comps = pca.fit_transform(all_embs)
                pc_normal_train = pri_comps[:norm_len]
                pc_sleepy_train = pri_comps[norm_len:]
                first = False
            else:
                pc_normal_train = avg_norm_embs
                pc_sleepy_train = avg_sleep_embs
                first = False
        else:
            norm_embs, sleep_embs = load_embeddings(subject)
            avg_norm_embs = embeddings_segment(norm_embs, len(norm_embs), seg_len)
            avg_sleep_embs = embeddings_segment(sleep_embs, len(sleep_embs), seg_len)
            if pca:
                norm_len = len(avg_norm_embs)
                sleep_len = len(avg_sleep_embs)
                all_embs = list(avg_norm_embs) + list(avg_sleep_embs)
                pca = PCA(n_components=3)
                pri_comps = pca.fit_transform(all_embs)
                new_pc_normal_train = pri_comps[:norm_len]
                new_pc_sleepy_train = pri_comps[norm_len:]
                pc_normal_train = np.concatenate((pc_normal_train, np.array(new_pc_normal_train)), axis=0)
                pc_sleepy_train = np.concatenate((pc_sleepy_train, np.array(new_pc_sleepy_train)), axis=0)
            else:
                pc_normal_train = np.concatenate((pc_normal_train, np.array(avg_norm_embs)), axis=0)
                pc_sleepy_train = np.concatenate((pc_sleepy_train, np.array(avg_sleep_embs)), axis=0)
                
    return pc_normal_train, pc_sleepy_train

def multi_subjects_class(subjects, test_subjects):
    subjects = list(map(str, subjects))
    num_test_subs = int(0.3 * len(subjects))
    
    if test_subjects:
        test_subs = list(map(str, test_subjects))
    else:
        test_subs = random.sample(subjects, k=num_test_subs)
    train_subs = list((Counter(subjects)-Counter(test_subs)).elements())
    
    print(train_subs)
    print(test_subs)
    
    X_normal_train, X_sleepy_train = obtain_data(train_subs, pca=True)
    X_normal_test, X_sleepy_test = obtain_data(test_subs, pca=True)
   

    y_normal_train = list(np.zeros(len(X_normal_train)))
    y_normal_test = list(np.zeros(len(X_normal_test)))

    y_sleepy_train = list(np.ones(len(X_sleepy_train)))
    y_sleepy_test = list(np.ones(len(X_sleepy_test)))


    X_normal = X_normal_train.tolist() + X_normal_test.tolist()
    X_sleepy = X_sleepy_train.tolist() + X_sleepy_test.tolist()

    X = X_normal + X_sleepy

    train_normal_end = len(X_normal_train)
    test_normal_end = len(X_normal_test) + train_normal_end

    train_sleepy_end = len(X_sleepy_train) + test_normal_end

    scaler = MinMaxScaler()
    X_norm = scaler.fit_transform(X)

    norm_X_normal_train = X_norm[:train_normal_end]
    norm_X_normal_test = X_norm[train_normal_end:test_normal_end]
    norm_X_sleepy_train = X_norm[test_normal_end:train_sleepy_end]
    norm_X_sleepy_test = X_norm[train_sleepy_end:]

    print(len(X_normal_train), len(norm_X_normal_train))
    print(len(X_normal_test), len(norm_X_normal_test))
    print(len(X_sleepy_train), len(norm_X_sleepy_train))
    print(len(X_sleepy_test), len(norm_X_sleepy_test))

    y_train = y_normal_train + y_sleepy_train
    y_test = y_normal_test + y_sleepy_test

    X_train = norm_X_normal_train.tolist() + norm_X_sleepy_train.tolist()
    X_test = norm_X_normal_test.tolist() + norm_X_sleepy_test.tolist()

    # train model
    model = SVC(kernel='linear')
    model.fit(np.array(X_train), y_train)

    y_pred = model.predict(np.array(X_test))
    f1 = f1_score(y_test, y_pred)
    f1_round =  round(f1, 2) 
    print("F1 score: ", f1_round)
    return model, f1_round, X_train, X_test, y_train, y_test

def k_fold_class(subjects, k, test_subjects=None):
    sum_f1 = 0
    for i in range(k):
        model, f1_round, X_train, X_test, y_train, y_test = multi_subjects_class(subjects, test_subjects)
        sum_f1 += f1_round
    return sum_f1 / k

#subjects = [1,3,4,6,7,9,10,11,12,14,15,16,20,22,23,24,25]
subjects = [9,10, 11, 12, 14,15,16,20,23,24, 25]
test_subjects = [10, 15, 12]

#model, f1_round, X_train, X_test, y_train, y_test = multi_subjects_class(subjects)
f1 = k_fold_class(subjects, 1, test_subjects)
print(f1)

## Temporal analysis

In [ ]:
def gradient_plot(comp1, comp2, condition, time, size=40, text = False):
    plt.title("Deep features of " + time + " min segments for subject " + subject[-2:] + ' in ' + condition + ' condition')
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.scatter(comp1, comp2, label = condition, cmap = 'Blues', c = range(len(comp1)), s=size)
    if text:
        for i in range(len(comp1)):
            plt.annotate(i + 1, (comp1[i], comp2[i]), fontsize = 20)

    plt.savefig('../final_figures/deep_gradient_scatter_' + subject + condition + time + '_min.jpg')
    plt.show()

def average(values, avg_len):
    new_values = []
    sum_values = []
    for i, val in enumerate(values):
        sum_values.append(val)
        if i % avg_len == 0 and i > 0:
            new_values.append(sum(sum_values) / len(sum_values))
            sum_values = []
            
    # if there are rest values
    if len(sum_values) > 0:
        new_values.append(sum(sum_values) / len(sum_values))
    return new_values

avg_len = 10
avg_comp1_normal = average(pc_normal[:,0], avg_len)
avg_comp2_normal = average(pc_normal[:,1], avg_len)

gradient_plot(avg_comp1_normal, avg_comp2_normal, 'normal', str(avg_len), size=180, text = True)
gradient_plot(pc_normal[:,0], pc_normal[:,1], 'normal', '1')
plt.show()

avg_comp1_sleepy = average(pc_sleep[:,0], avg_len)
avg_comp2_sleepy = average(pc_sleep[:,1], avg_len)

gradient_plot(avg_comp1_sleepy, avg_comp2_sleepy, 'sleepy', str(avg_len), size=180, text = True)
gradient_plot(pc_sleep[:,0], pc_sleep[:,1], 'sleepy', '1')
plt.show()

## Clustering techniques
### DBSCAN

In [ ]:
def plot_clusters(embs, pri_comps, epsilon, cond):
    cluster_model = DBSCAN(eps = epsilon, min_samples = 1).fit(np.array(embs))
    labels = cluster_model.labels_
    uni_labels = np.unique(labels)
    print(labels)

    all_values = []
    for label in uni_labels:
        label_values = []
        for i, emb in enumerate(pri_comps):
            if labels[i] == label:
                label_values.append(list(pri_comps[i]))
        plt.scatter(np.array(label_values)[:,0], np.array(label_values)[:,1], label = str(label))

    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('DBSCAN Clustering of 1 min segments of subject ' + subject[-2:]  + ' ' + cond + ' condition')
    plt.legend()

    plt.savefig('../final_figures/clusters_' + subject + '_' + cond + '.jpg')
    plt.show()
    
    return labels

#all_labels = plot_clusters(all_embs, pri_comps, 0.3, "all")
normal_labels = plot_clusters(avg_norm_embs, pc_normal, 0.10, "normal")
sleep_labels = plot_clusters(avg_sleep_embs, pc_sleep, 0.22, "sleepy")

### Evolution of clusters trough time

In [ ]:
def make_barplot(labels, cond):
    occurences = {}
    y_values = []
    
    all_colors = ['blue', 'orange', 'green', 'red', 'cyan', 'yellow', 'purple', 'olive', 'pink']
    colors = []
    
    for lab in labels:
        if lab in occurences:
            occurences[lab] += 1
        else:
            occurences[lab] = 1
        y_values.append(occurences[lab])
        colors.append(all_colors[lab])
    x_values = range(len(labels))
    
    plt.bar(x_values, y_values, color=colors)
    plt.title("Evolution of clusters of subject " + subject[-2:] + " " + cond + " condition using DBSCAN")
    plt.xlabel("Time in Minutes")
    plt.ylabel("Size of clusters")
    plt.savefig("../final_figures/clusters_evolution_" + subject + "_" + cond + ".jpg")
    plt.show()

#make_barplot(all_labels, "all")
make_barplot(normal_labels, "normal")
make_barplot(sleep_labels, "sleepy")

In [ ]:
from sklearn.manifold import TSNE
tsne_data = TSNE(perplexity = 20 ).fit_transform(np.array(all_embs))

tsne_normal = tsne_data[:norm_len]
tsne_sleep = tsne_data[norm_len:]
print(len(tsne_normal))

In [ ]:
plt.scatter(tsne_normal[:,0], tsne_normal[:,1], label = 'Normal')
plt.scatter(tsne_sleep[:,0], tsne_sleep[:,1], label = 'Sleep Restricted')

plt.xlabel('T-SNE Component 1')
plt.ylabel('T-SNE Component 2')
plt.title('Visualizing 1 min segments of subject ' + subject[-2:] + ' using t-SNE')
plt.legend()
plt.savefig('../final_figures/tsne_plot_subject' + subject[-2:] + '.jpg')
plt.show()

In [ ]:
cluster_model = DBSCAN(eps = 0.28, min_samples = 1).fit(np.array(all_embs))
labels = cluster_model.labels_
uni_labels = np.unique(labels)
print(labels)

all_values = []
for label in uni_labels:
    label_values = []
    for i, emb in enumerate(tsne_data):
        if labels[i] == label:
            label_values.append(list(tsne_data[i]))
    plt.scatter(np.array(label_values)[:,0], np.array(label_values)[:,1], label = str(label))

plt.xlabel('T-sne Component 1')
plt.ylabel('T-sne Component 2')
plt.title('Clusters of 1 min segments of subject 10')
plt.legend()

plt.savefig('../figures/clusters_sub10_tsne.jpg')
plt.show()

## Multiple subjects analysis

In [ ]:
norm_embs10, sleep_embs10 = load_embeddings('sub10')
norm_embs9, sleep_embs9 = load_embeddings('sub9')

avg_norm_embs10 = embeddings_segment(norm_embs10, len(norm_embs10), 46 * 60)
avg_sleep_embs10 = embeddings_segment(sleep_embs10, len(sleep_embs10), 46 * 60)

avg_norm_embs9 = embeddings_segment(norm_embs9, len(norm_embs9), 46 * 60)
avg_sleep_embs9 = embeddings_segment(sleep_embs9, len(sleep_embs9), 46 * 60)

print(len(norm_embs), len(sleep_embs))

all_embs = avg_norm_embs9 + avg_sleep_embs9 + avg_norm_embs10 + avg_sleep_embs10
norm_len9 = len(avg_norm_embs9)
sleep_len9 = norm_len9 + len(avg_sleep_embs9)
norm_len10 = sleep_len9 + len(avg_norm_embs10)
sleep_len10 = norm_len10 + len(avg_sleep_embs10)
print(len(all_embs))
print(sleep_len10)

pca = PCA(n_components=2)
pri_comps_multi = pca.fit_transform(all_embs)

pc_normal9 = pri_comps_multi[:norm_len9]
pc_sleep9 = pri_comps_multi[norm_len9: sleep_len9]
pc_normal10 = pri_comps_multi[sleep_len9:norm_len10]
pc_sleep10 = pri_comps_multi[norm_len10:]

print(len(pc_normal9) + len(pc_sleep9) + len(pc_normal10) + len(pc_sleep10))

In [ ]:
plt.scatter(pc_normal9[:,0], pc_normal9[:,1], label = 'Normal Subject 9')
plt.scatter(pc_sleep9[:,0], pc_sleep9[:,1], label = 'Sleep Restricted Subject 9')
plt.scatter(pc_normal10[:,0], pc_normal10[:,1], label = 'Normal Subject 10')
plt.scatter(pc_sleep10[:,0], pc_sleep10[:,1], label = 'Sleep Restricted Subject 10')
#plt.scatter(pri_comps[:,0], pri_comps[:,1])

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('The first two principal components of 1 min segments of subject 9 and 10')
plt.legend()
plt.savefig('pc_plot_subj9&10.jpg')
plt.show()

### Textfile

In [ ]:
import os

def make_textfile(embeddings):
    with open('../textfiles/' + subject + '/' + subject + '_1', 'w') as f:
        for row in embeddings:
            first = True
            for value in row:
                if not first:
                    f.write(';')
                else:
                    first = False
                f.write(str(value))
            f.write('\n')
    print('done')
make_textfile(np.array(avg_norm_embs))
print(np.array(avg_norm_embs).shape)